In [13]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from plotly.colors import sequential
from scipy.optimize import curve_fit
from scipy.stats import chi2

In [14]:
def read(filename):
    df = pd.read_csv(filename, skiprows = 2, nrows = 151, sep = ',', header = None)
    df.dropna(axis = 1, inplace = True)  # Drop columns with NaN values
    df.reset_index(drop = True, inplace = True)  # Reset index
    df.columns = ['w0', 'I0', 'w1', 'I1', 'w2', 'I2', 'w3', 'I3', 'w4', 'I4', 'w5', 'I5']
    return df

In [15]:
def plottiamo(data):
    fig = go.Figure()
    colors = sequential.Sunsetdark_r[:6]

    for i in range(6):
        fig.add_trace(go.Scatter(
            x = data[f'w{i}'], 
            y = data[f'I{i}'], 
            mode = 'lines', 
            name = f'M {i}',
            line = dict(color = colors[i])
        ))

    fig.update_layout(
        xaxis_title = "Wavelength (nm)",
        yaxis_title = "Intensity",
        template = "plotly_white",
        height = 500,
        width = 800
    )

    fig.show()

In [16]:
file = 'fluo2/GuHCl5.csv'
data = read(file)
plottiamo(data)

In [17]:
def parabola(x, a, x0, yV):
    return -a * (x - x0)**2 + yV

In [18]:
fit_results, fit_errors = {}, {}
V = []
colors = sequential.Sunsetdark_r[:6]  # Colori per le serie
SIGMA_LAMBDA = 1.5  # nm DICHIARATI DAL COSTRUTTORE
POINTS_AROUND_MAX = 5  # Numero di punti attorno al massimo per il fit

fig = go.Figure()

for i in range(6):
    w_col, I_col = f'w{i}', f'I{i}'
    fig.add_trace(go.Scatter(
        x=data[w_col], y=data[I_col], mode='lines', name=f'{i} M',
        line=dict(color=colors[i], dash='dot')
    ))

    # Fit parabolico
    max_idx = data[I_col].idxmax()
    l_idx = max(0, max_idx - POINTS_AROUND_MAX // 2)
    r_idx = min(len(data) - 1, max_idx + POINTS_AROUND_MAX // 2)  # range di indici per il fit

    w_subset, I_subset = data[w_col].iloc[l_idx:r_idx + 1], data[I_col].iloc[l_idx:r_idx + 1]  # subset creato con gli indici nel range

    # Iterazione 0
    popt, pcov = curve_fit(parabola, w_subset, I_subset, p0=[1, w_subset.mean(), I_subset.max()])
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))

    # Iterazione 1
    sy = np.full_like(w_subset, SIGMA_LAMBDA)  # Errore su x (sigma lambda)
    popt, pcov = curve_fit(parabola, w_subset, I_subset, p0=[a, λcenter, IMAX], sigma=sy)
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))

    # Salvataggio dei risultati del fit
    fit_results[f'{i} M'] = popt
    fit_errors[f'{i} M'] = [error_λcenter, error_a, error_IMAX]

    # Calcolo del vertice
    a, xV, yV = popt
    V.append([xV, yV])
    x_parabola = np.linspace(w_subset.min(), w_subset.max(), 100)
    y_parabola = parabola(x_parabola, *popt)

    # Aggiunta della parabola
    fig.add_trace(go.Scatter(
        x=x_parabola, y=y_parabola, mode='lines', name=f'Parabola Fit (Series {i})',
        line=dict(color=colors[i]), showlegend=False
    ))

    # Vertice con barre d'errore
    fig.add_trace(go.Scatter(
        x=[xV], y=[yV], mode='markers', name=f'Vertex (Series {i})',
        marker=dict(color=colors[i], size=8, symbol='circle'),
        showlegend=False
    ))

# Configurazione del layout
fig.update_layout(
    xaxis_title="Wavelength (nm)",
    yaxis_title="Intensity",
    template="plotly_white",
    height=500,
    width=800
)

fig.show()

# Stampa dei risultati del fit
for key, values in fit_results.items():
    errors = fit_errors[key]
    formatted_values = f"xV = {values[1]:.2f} ± {errors[1]:.2f}, yV = {values[2]:.2f} ± {errors[2]:.2f}"
    print(f"{key}: {formatted_values}")


RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 800.

In [7]:
# Cden = concentrazione del denaturante -> valore x
# Cmp = concentrazione del campione -> parametro
# m = pendenza della curva -> parametro
# yN = intensità del campione non denaturato -> valore noto
# yD = intensità del campione denaturato -> valore noto

def sigmoide(Cden, Cmp, m): 
    R = 8.314
    T = 298.15
    yN = V[0][0]
    yD = V[5][0]
    expon = np.exp(-m * (Cmp - Cden) / (R * T))
    return (yN + yD * expon) / (1 + expon)

In [8]:
# Estrai gli errori su xV dai dati esistenti
errors_xV = [fit_errors[f'{i} M'][1] for i in range(6)]  # Errori su xV per ogni serie

# Definizione di assex e assey dai dati dei vertici
assex = [i for i in range(6)]  # Indici delle serie (0 M, 1 M, ..., 5 M)
assey = [V[i][0] for i in range(6)]  # Valori di xV per ogni serie

# Fit dei dati usando la funzione sigmoide e considerando gli errori
popt1, pcov1 = curve_fit(sigmoide, assex, assey, sigma=errors_xV, p0=[1, 1], maxfev=5000)


In [9]:
fig = go.Figure()

# Fit sigmoide
Cden_fit = np.linspace(min(assex), max(assex), 100)  # Genera valori per l'asse x
sigmoide_fit = sigmoide(Cden_fit, *popt1)  # Calcola i valori della sigmoide

# Sigmoide grafico
fig.add_trace(go.Scatter(
    x=Cden_fit,
    y=sigmoide_fit,
    mode='lines',
    name='Fit Sigmoide',
    line=dict(color=colors[0]),
    showlegend=False
))

# Dati originali con barre d'errore
fig.add_trace(go.Scatter(
    x=assex,
    y=assey,
    mode='markers',
    name='Dati Originali',
    marker=dict(color=colors[2], size=5),
    error_y=dict(
        type='data',
        array=errors_xV,
        visible=True
    ),
    showlegend=False
))

fig.update_layout(
    xaxis_title="C denaturante (M)",
    yaxis_title="wavelength (nm)",
    template="plotly_white",
    height=500,
    width=800    
)

fig.show()

In [10]:

y_fit = sigmoide(np.array(assex), *popt1)
errors_y = errors_xV  # Errori associati ai dati osservati

ChiQ = np.sum(((np.array(assey) - y_fit) / np.array(errors_y))**2)
dof = len(assey) - len(popt1)
rChiQ = ChiQ / dof
p_value = 1 - chi2.cdf(ChiQ, dof)

print(f"Chi quadro ridotto: {rChiQ:.2f}")
print(f"P-value: {p_value:.4f}")

Chi quadro ridotto: 1.91
P-value: 0.1054
